# Comparing GDP to Educational Attainment

The purpose of this project is to determine whether there is a correlation between GDP and school life expectancy. Is the wealth of a country an indicator of how long its students remain in school? 

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from bs4 import BeautifulSoup
import requests
import pandas as pd
import math
import statsmodels.formula.api as smf

The first step is to scrape the education data from the United Nations Statistics Devision website. The data below divides school life expectancy into two groups, male and female.

In [4]:
url = "http://web.archive.org/web/20110514112442/http://unstats.un.org/unsd/demographic/products/socind/education.htm"

r = requests.get(url)

soup = BeautifulSoup(r.content, 'lxml')

for row in soup('table'):
    print(row)

<table style="width:100%;"><tbody><tr>
<td id="wm-logo">
<a href="/web/" title="Wayback Machine home page"><img alt="Wayback Machine" border="0" height="39" src="/static/images/toolbar/wayback-toolbar-logo.png" width="110"/></a>
</td>
<td class="c">
<table style="margin:0 auto;"><tbody><tr>
<td class="u" colspan="2">
<form action="/web/form-submit.jsp" id="wmtb" method="get" name="wmtb" target="_top"><input id="wmtbURL" name="url" onfocus="this.focus();this.select();" style="width:400px;" type="text" value="http://unstats.un.org/unsd/demographic/products/socind/education.htm"/><input name="type" type="hidden" value="replay"/><input name="date" type="hidden" value="20110514112442"/><input type="submit" value="Go"/><span id="wm_tb_options" style="display:block;"></span></form>
</td>
<td class="n" rowspan="2">
<table><tbody>
<!-- NEXT/PREV MONTH NAV AND MONTH INDICATOR -->
<tr class="m">
<td class="b" nowrap="nowrap">
<a href="/web/20100819143645/http://unstats.un.org/unsd/demographic/pro

In [5]:
data = soup('table')[6]
print(data)

<table border="0" cellpadding="0" cellspacing="0" width="800">
<tr valign="top">
<td><!-- #BeginEditable "MainContent" -->
<table align="center" border="0" width="540">
<tr>
<td cllcontsubhd="" colspan="2" valign="bottom"><h2 align="center"><a name="Top"></a>Social 
              indicators </h2></td>
</tr>
<tr>
<td cllcontsubhd="" colspan="2" height="23" valign="top"><div align="center" class="csubhd"><a href="default.htm">Introduction</a> 
              | <a href="statistics.htm">Statistics</a></div></td>
</tr>
</table>
<table cellpadding="0" cellspacing="0">
<tr>
<td height="1" width="797"><div align="right">
<p class="Clear"><a href="Education_Dec2010.xls" style="color: #666; font-weight: bold">Download data</a> <a href="Education_Dec2010.xls"><img alt="Excel" border="0" height="21" name="Excel" src="/web/20110514112442im_/http://unstats.un.org/unsd/img/excel.gif" width="22"/></a></p>
<div align="left">
<table align="left" cellpadding="0" cellspacing="0">
<tr class="bar1">
<td cols

In [6]:
data.find_all("td", height="19", text=True)

[<td height="19" width="281"><span style="color: #336699">Country or area</span></td>,
 <td height="19" width="87"><div align="right" style="color: #336699">Year</div></td>,
 <td colspan="3" height="19"><div align="center" style="color: #336699">Total</div></td>,
 <td colspan="3" height="19"><div align="center" style="color: #336699">Men</div></td>,
 <td colspan="2" height="19"><div align="center" style="color: #336699">Women</div></td>,
 <td height="19">Afghanistan</td>,
 <td align="right" height="19">2004</td>,
 <td height="19"> </td>,
 <td height="19">a</td>,
 <td align="right" height="19" width="71">8</td>,
 <td align="right" height="19" width="72">11</td>,
 <td align="right" height="19" width="64">5</td>,
 <td height="19">Albania</td>,
 <td align="right" height="19">2004</td>,
 <td height="19"> </td>,
 <td height="19">a</td>,
 <td align="right" height="19">11</td>,
 <td align="right" height="19">11</td>,
 <td align="right" height="19">11</td>,
 <td height="19">Algeria</td>,
 <td a

In [103]:
lzt = []
for node in data.find_all("td", height="19", text=True):
     lzt.append(''.join(node.find_all(text=True)))
lzt = [i for i in lzt if i not in (u'\xa0',u'a',u'b',u'c',u'd',u'e',u'f',u'g',u'h')]
lzt2 = [lzt[i:i+5] for i in range(0, len(lzt), 5)]
df = pd.DataFrame(lzt2)
df.columns = df.iloc[0]
df_final = df[1:]
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 1 to 186
Data columns (total 5 columns):
Country or area    186 non-null object
Year               185 non-null object
Total              185 non-null object
Men                185 non-null object
Women              185 non-null object
dtypes: object(5)
memory usage: 7.3+ KB


In [104]:
df_final.head()

,Country or area,Year,Total,Men,Women
1,Afghanistan,2004,8,11,5
2,Albania,2004,11,11,11
3,Algeria,2005,13,13,13
4,Andorra,2008,11,11,12
5,Anguilla,2008,11,11,11


In [105]:
df_final.tail()

,Country or area,Year,Total,Men,Women
182,Zambia,2000,7,8,7
183,Zimbabwe,2003,9,9,9
184,...,Data not available.,UIS Estimation.,b,National Estimation.
185,Data refers to 2005.,Data refers to 2009.,Data refers to 2006.,Data refers to 2008.,Data refers to 2007.
186,Data refers to 2001.,None,None,None,None


In [106]:
df_final = df[1:184]
df_final.tail()

,Country or area,Year,Total,Men,Women
179,Venezuela (Bolivarian Republic of),2008,14,13,15
180,Viet Nam,2001,10,11,10
181,Yemen,2005,9,11,7
182,Zambia,2000,7,8,7
183,Zimbabwe,2003,9,9,9


In [109]:
df_final[['Total','Men','Women']] = df_final[['Total','Men','Women']].astype(int)

C:\Users\Matt\Anaconda3\lib\site-packages\pandas\core\frame.py:2369: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [110]:
df_final.describe()

,Total,Men,Women
count,183.000000,183.000000,183.000000
mean,12.448087,12.393443,12.513661
std,3.053782,2.668160,3.683730
min,5.000000,5.000000,4.000000
25%,10.500000,11.000000,10.000000
50%,13.000000,12.000000,13.000000
75%,15.000000,14.000000,15.000000
max,21.000000,20.000000,21.000000
